# Generative AI Media Entertainment Workshop
## Prompt Engineering with Amazon Bedrock

> *This notebook should work well with the **`Python 3`** kernel in SageMaker Studio*

---

In this demo notebook, we demonstrate how to use the [`boto3` Python SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) to work with [Amazon Bedrock](https://aws.amazon.com/bedrock/) Foundation Models.

---

## Prerequisites

Run the cells in this section to install the packages needed by the notebooks in this workshop. ⚠️ You will see pip dependency errors, you can safely ignore these errors. ⚠️

IGNORE ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"


---

## Create the boto3 client

Interaction with the Bedrock API is done via the AWS SDK for Python: [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).

#### Use different clients
The boto3 provides different clients for Amazon Bedrock to perform different actions. The actions for [`InvokeModel`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModel.html) and [`InvokeModelWithResponseStream`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModelWithResponseStream.html) are supported by Amazon Bedrock Runtime where as other operations, such as [ListFoundationModels](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_ListFoundationModels.html), are handled via [Amazon Bedrock client](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_Operations_Amazon_Bedrock.html).


#### Use the default credential chain

If you are running this notebook from [Amazon Sagemaker Studio](https://aws.amazon.com/sagemaker/studio/) and your Sagemaker Studio [execution role](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) has permissions to access Bedrock you can just run the cells below as-is. This is also the case if you are running these notebooks from a computer whose default AWS credentials have access to Bedrock.



In [ ]:
import json
import os
import sys

import boto3

boto3_bedrock = boto3.client('bedrock')

#### Validate the connection

We can check the client works by trying out the `list_foundation_models()` method, which will tell us all the models available for us to use 

In [ ]:
boto3_bedrock.list_foundation_models()


---

## Lab 1 - Text Summarization in Bedrock

Picture yourself at a movie studio, or maybe a video streaming company. You get synopses all the time, and hardly have time to read them. In this Lab you will use Foundation Models (FMs) in Amazon Bedrock to summarize synopsis, so it is much easier to read and digest.

Here is a [Synopsis for the movie Whiplash](https://www.scriptreaderpro.com/wp-content/uploads/2019/07/Film-Synopsis-Example-Whiplash.pdf). We will use it below.



In [ ]:
import boto3
import botocore
import json 

bedrock_runtime = boto3.client('bedrock-runtime')


### Amazon Titan Large

In [ ]:
prompt_data = """
Summarize the following text:
<text>
Andrew Neiman, a young jazz student at the Shaffer Conservatory in New York, has one dream:
to go down in history as one of the world’s best drummers. He’s therefore thrilled when Terence
Fletcher, a famous conductor, invites him to join the conservatory’s Studio Band as a core
alternate drummer. Fletcher, however, turns out to be anything but an ordinary teacher. He’s a
sadistic tyrant and Andrew realizes just how much of one when he has a chair hurled at him for
failing to keep time.
At a jazz competition, Andrew misplaces the sheet music to “Whiplash,” meaning their core
drummer can’t play. Andrew, however, can—from memory—and after a first class performance,
Fletcher promotes him to core drummer. But Andrew’s joy won’t last long… In a typically
twisted move, Fletcher bumps Andrew back down to alternate drummer, putting a much lesstalented musician in his place. More determined than ever, Andrew breaks up with his girlfriend
and practices until his hands bleed. It pays off… After a grueling five-hour audition, during
which Fletcher kicks furniture and screams at him, Andrew earns back the core spot.
Andrew arrives late for another competition after his bus breaks down, hires a car, then realizes
he left his drumsticks at the car rental office. He races back, retrieves them, but on his way to
the theater, his car is broadsided by a semi. He crawls from the wreckage and runs the rest of
the way, finally arriving on stage bloody and injured. When he struggles to play, Fletcher cooly
dismisses him. Enraged, Andrew attacks Fletcher in front of the audience, which gets him
dismissed from the school.
Andrew files an ethics complaint against Shaffer Conservatory and learns that one of Fletcher’s
former students hanged himself due to his emotional and physical abuse. Andrew agrees to
testify as an anonymous witness and Fletcher is fired. Andrew gives up drumming and, months
later, stumbles upon Fletcher playing piano in a jazz club. They go for a drink, during which
Fletcher explains why he pushed his students so hard: so that they might become the next
Charlie Parker. In Fletcher’s eyes the greats like Parker wouldn’t be discouraged by anything.
He then invites Andrew to drum with his band at a jazz festival. Has Fletcher changed? Andrew
thinks so, and accepts.
On stage at the festival, Fletcher has two surprises for Andrew. One: he knows he testified
against him, and two: they’re starting with a piece Andrew doesn’t know and for which there’s
no sheet music. Unable to play, Andrew leaves the stage humiliated. But he returns, interrupts
Fletcher and cues the band, before launching into a breathtaking solo. Fletcher is taken aback,
but in that moment realizes the enormity of Andrew’s talent and begins to guide him. As
Andrew ends his solo, they share a smile and Fletcher cues the finale.
</text>

"""

Next, we will construct the body and response to pass the prompt above to the Cohere Command text model.

In [ ]:
try:

    body = json.dumps({
      "prompt": prompt_data,
      "max_tokens":400,
      "temperature":0.2 
    })

    response = bedrock_runtime.invoke_model(
      body=body,
	  modelId="cohere.command-text-v14", 
      accept= "*/*", 
      contentType="application/json"
    )

    response_body = json.loads(response.get('body').read())
    parse_text = response_body['generations'][0]['text']
    parse_text

    print(parse_text)
    
except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

## Refine the output

This is good, but there are ways to refine this result.

## Prompt engineering
Prompt engineering is a discipline focused on developing optimized prompts to efficiently apply language models to various tasks.

Try the same synopsis, but this time followed by "Summarize the text above in one sentence:".

In [ ]:
prompt_data = """
Summarize the following text in one sentence:
<text>
Andrew Neiman, a young jazz student at the Shaffer Conservatory in New York, has one dream:
to go down in history as one of the world’s best drummers. He’s therefore thrilled when Terence
Fletcher, a famous conductor, invites him to join the conservatory’s Studio Band as a core
alternate drummer. Fletcher, however, turns out to be anything but an ordinary teacher. He’s a
sadistic tyrant and Andrew realizes just how much of one when he has a chair hurled at him for
failing to keep time.
At a jazz competition, Andrew misplaces the sheet music to “Whiplash,” meaning their core
drummer can’t play. Andrew, however, can—from memory—and after a first class performance,
Fletcher promotes him to core drummer. But Andrew’s joy won’t last long… In a typically
twisted move, Fletcher bumps Andrew back down to alternate drummer, putting a much lesstalented musician in his place. More determined than ever, Andrew breaks up with his girlfriend
and practices until his hands bleed. It pays off… After a grueling five-hour audition, during
which Fletcher kicks furniture and screams at him, Andrew earns back the core spot.
Andrew arrives late for another competition after his bus breaks down, hires a car, then realizes
he left his drumsticks at the car rental office. He races back, retrieves them, but on his way to
the theater, his car is broadsided by a semi. He crawls from the wreckage and runs the rest of
the way, finally arriving on stage bloody and injured. When he struggles to play, Fletcher cooly
dismisses him. Enraged, Andrew attacks Fletcher in front of the audience, which gets him
dismissed from the school.
Andrew files an ethics complaint against Shaffer Conservatory and learns that one of Fletcher’s
former students hanged himself due to his emotional and physical abuse. Andrew agrees to
testify as an anonymous witness and Fletcher is fired. Andrew gives up drumming and, months
later, stumbles upon Fletcher playing piano in a jazz club. They go for a drink, during which
Fletcher explains why he pushed his students so hard: so that they might become the next
Charlie Parker. In Fletcher’s eyes the greats like Parker wouldn’t be discouraged by anything.
He then invites Andrew to drum with his band at a jazz festival. Has Fletcher changed? Andrew
thinks so, and accepts.
On stage at the festival, Fletcher has two surprises for Andrew. One: he knows he testified
against him, and two: they’re starting with a piece Andrew doesn’t know and for which there’s
no sheet music. Unable to play, Andrew leaves the stage humiliated. But he returns, interrupts
Fletcher and cues the band, before launching into a breathtaking solo. Fletcher is taken aback,
but in that moment realizes the enormity of Andrew’s talent and begins to guide him. As
Andrew ends his solo, they share a smile and Fletcher cues the finale.
</text>

"""

In [ ]:
try:

    body = json.dumps({
      "prompt": prompt_data,
      "max_tokens":400,
      "temperature":0.2 
    })

    response = bedrock_runtime.invoke_model(
      body=body,
	  modelId="cohere.command-text-v14", 
      accept= "*/*", 
      contentType="application/json"
    )

    response_body = json.loads(response.get('body').read())
    parse_text = response_body['generations'][0]['text']
    parse_text

    print(parse_text)
    
except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

## Your turn 

Modify the prompt below to output the summary into one paragraph with output similar to the one below:

_Andrew Neiman is a young drummer who dreams of greatness. He joins the Shaffer Conservatory in New York and is invited to join a band led by the conductor Terence Fletcher. However, Fletcher turns out to be a tyrannical and sadistic bandleader who abuses his students. Despite this, Andrew is determined to succeed and works hard to impress Fletcher. He is eventually promoted to core drummer but is later demoted. This motivates Andrew to practice even harder, and he eventually earns back his core spot. However, Fletcher continues to abuse him, and Andrew eventually testifies against him, leading to Fletcher's dismissal from the school. Months later, Andrew encounters Fletcher again and decides to give him a chance, but Fletcher retaliates by setting up a situation where Andrew fails again. However, Andrew manages to turn the situation around and impresses Fletcher in the end._

In [ ]:
prompt_data = """
Add your prompt here:
<text>
Andrew Neiman, a young jazz student at the Shaffer Conservatory in New York, has one dream:
to go down in history as one of the world’s best drummers. He’s therefore thrilled when Terence
Fletcher, a famous conductor, invites him to join the conservatory’s Studio Band as a core
alternate drummer. Fletcher, however, turns out to be anything but an ordinary teacher. He’s a
sadistic tyrant and Andrew realizes just how much of one when he has a chair hurled at him for
failing to keep time.
At a jazz competition, Andrew misplaces the sheet music to “Whiplash,” meaning their core
drummer can’t play. Andrew, however, can—from memory—and after a first class performance,
Fletcher promotes him to core drummer. But Andrew’s joy won’t last long… In a typically
twisted move, Fletcher bumps Andrew back down to alternate drummer, putting a much lesstalented musician in his place. More determined than ever, Andrew breaks up with his girlfriend
and practices until his hands bleed. It pays off… After a grueling five-hour audition, during
which Fletcher kicks furniture and screams at him, Andrew earns back the core spot.
Andrew arrives late for another competition after his bus breaks down, hires a car, then realizes
he left his drumsticks at the car rental office. He races back, retrieves them, but on his way to
the theater, his car is broadsided by a semi. He crawls from the wreckage and runs the rest of
the way, finally arriving on stage bloody and injured. When he struggles to play, Fletcher cooly
dismisses him. Enraged, Andrew attacks Fletcher in front of the audience, which gets him
dismissed from the school.
Andrew files an ethics complaint against Shaffer Conservatory and learns that one of Fletcher’s
former students hanged himself due to his emotional and physical abuse. Andrew agrees to
testify as an anonymous witness and Fletcher is fired. Andrew gives up drumming and, months
later, stumbles upon Fletcher playing piano in a jazz club. They go for a drink, during which
Fletcher explains why he pushed his students so hard: so that they might become the next
Charlie Parker. In Fletcher’s eyes the greats like Parker wouldn’t be discouraged by anything.
He then invites Andrew to drum with his band at a jazz festival. Has Fletcher changed? Andrew
thinks so, and accepts.
On stage at the festival, Fletcher has two surprises for Andrew. One: he knows he testified
against him, and two: they’re starting with a piece Andrew doesn’t know and for which there’s
no sheet music. Unable to play, Andrew leaves the stage humiliated. But he returns, interrupts
Fletcher and cues the band, before launching into a breathtaking solo. Fletcher is taken aback,
but in that moment realizes the enormity of Andrew’s talent and begins to guide him. As
Andrew ends his solo, they share a smile and Fletcher cues the finale.
</text>

"""

In [ ]:
try:

    body = json.dumps({
      "prompt": prompt_data,
      "max_tokens":400,
      "temperature":0.2 
    })

    response = bedrock_runtime.invoke_model(
      body=body,
	  modelId="cohere.command-text-v14", 
      accept= "*/*", 
      contentType="application/json"
    )

    response_body = json.loads(response.get('body').read())
    parse_text = response_body['generations'][0]['text']
    parse_text

    print(parse_text)
    
except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

## Changing prompt parameters.
Lets see what happens if we change the prompt parameters.

Below we are going to change the max_tokens to 50 lets see what happens.


In [ ]:
prompt_data = """
Summarize the following text as one paragraph:
<text>
Andrew Neiman, a young jazz student at the Shaffer Conservatory in New York, has one dream:
to go down in history as one of the world’s best drummers. He’s therefore thrilled when Terence
Fletcher, a famous conductor, invites him to join the conservatory’s Studio Band as a core
alternate drummer. Fletcher, however, turns out to be anything but an ordinary teacher. He’s a
sadistic tyrant and Andrew realizes just how much of one when he has a chair hurled at him for
failing to keep time.
At a jazz competition, Andrew misplaces the sheet music to “Whiplash,” meaning their core
drummer can’t play. Andrew, however, can—from memory—and after a first class performance,
Fletcher promotes him to core drummer. But Andrew’s joy won’t last long… In a typically
twisted move, Fletcher bumps Andrew back down to alternate drummer, putting a much lesstalented musician in his place. More determined than ever, Andrew breaks up with his girlfriend
and practices until his hands bleed. It pays off… After a grueling five-hour audition, during
which Fletcher kicks furniture and screams at him, Andrew earns back the core spot.
Andrew arrives late for another competition after his bus breaks down, hires a car, then realizes
he left his drumsticks at the car rental office. He races back, retrieves them, but on his way to
the theater, his car is broadsided by a semi. He crawls from the wreckage and runs the rest of
the way, finally arriving on stage bloody and injured. When he struggles to play, Fletcher cooly
dismisses him. Enraged, Andrew attacks Fletcher in front of the audience, which gets him
dismissed from the school.
Andrew files an ethics complaint against Shaffer Conservatory and learns that one of Fletcher’s
former students hanged himself due to his emotional and physical abuse. Andrew agrees to
testify as an anonymous witness and Fletcher is fired. Andrew gives up drumming and, months
later, stumbles upon Fletcher playing piano in a jazz club. They go for a drink, during which
Fletcher explains why he pushed his students so hard: so that they might become the next
Charlie Parker. In Fletcher’s eyes the greats like Parker wouldn’t be discouraged by anything.
He then invites Andrew to drum with his band at a jazz festival. Has Fletcher changed? Andrew
thinks so, and accepts.
On stage at the festival, Fletcher has two surprises for Andrew. One: he knows he testified
against him, and two: they’re starting with a piece Andrew doesn’t know and for which there’s
no sheet music. Unable to play, Andrew leaves the stage humiliated. But he returns, interrupts
Fletcher and cues the band, before launching into a breathtaking solo. Fletcher is taken aback,
but in that moment realizes the enormity of Andrew’s talent and begins to guide him. As
Andrew ends his solo, they share a smile and Fletcher cues the finale.
</text>

"""

In [ ]:
try:

    body = json.dumps({
      "prompt": prompt_data,
      "max_tokens":50,
      "temperature":0.2 
    })

    response = bedrock_runtime.invoke_model(
      body=body,
	  modelId="cohere.command-text-v14", 
      accept= "*/*", 
      contentType="application/json"
    )

    response_body = json.loads(response.get('body').read())
    parse_text = response_body['generations'][0]['text']
    parse_text

    print(parse_text)
    
except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

----
# Lab 2
